# Chapter 10. Proccesing text
## Notebook for R

### 10.1 Reading and cleaning text

In [1]:
install.packages(c("glue", "tidyverse"))

Installing packages into ‘/home/wva/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cli’, ‘processx’, ‘gargle’, ‘ids’, ‘rematch2’, ‘callr’, ‘broom’, ‘dbplyr’, ‘dtplyr’, ‘googledrive’, ‘googlesheets4’, ‘reprex’




In [2]:
library(glue)
library(tidyverse)

── Attaching packages ──────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse() masks glue::collapse()
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()



In [3]:
text = "This is text."
glue("class(text): {class(text)}")
print("length(text): {length(text)}")
glue("text[1]: {text[1]}")
glue("str_length(text): {str_length(text)}")
glue("str_sub(text, 6,7): {str_sub(text, 6,7)}")


class(text): character

[1] "length(text): {length(text)}"


text[1]: This is text.

str_length(text): 13

str_sub(text, 6,7): is

In [4]:
words = c("These", "are", "words")
glue("class(words): {class(words)}")
print("length(words): {length(words)}")
glue("words[1]: {words[1]}")
# Note: use collapse to convert to single value
words_2_3 = str_c(words[2:3], collapse=", ")
glue("words[2:3]: {words_2_3}")

class(words): character

[1] "length(words): {length(words)}"


words[1]: These

words[2:3]: are, words

In [5]:
text = "    <b>Communication</b>    
     (from Latin communicare, meaning to share)  "
cleaned = text %>% 
  # remove HTML tags:
  str_replace_all("<b>", " ")  %>% 
  str_replace_all("</b>", " ")  %>% 
  # normalize white space 
  str_squish() %>%
  # lower case
  tolower()  %>% 
  # trim spaces at start and end
  trimws()

glue(cleaned)

communication (from latin communicare, meaning to share)

In [6]:
text = "    <b>Communication</b>    
     (from Latin communicare, meaning to share)  "
cleaned = text %>% 
  # remove HTML tags:
  str_replace_all("<[^>]+>", " ")  %>% 
  # normalize white space 
  str_replace_all("\\p{space}+", " ")  %>% 
  # trim spaces at start and end
  str_remove_all("^\\s+|\\s+$")

cleaned

[1] "Communication (from Latin communicare, meaning to share)"

In [7]:
library(tidyverse)
url="https://cssbook.net/d/example_tweets.csv"
tweets = read_csv(url)
tweets = tweets %>% mutate(
    # identify tweets with hashtags
    has_tag=str_detect(text, "#\\w+"),
    # How many at-mentions are there?
    n_at = str_count(text, "(^|\\s)@\\w+"),
    # Extract first url
    url = str_extract(text, "(https?://\\S+)"),
    # Remove at-mentions, tags, and urls
    plain2 = str_replace_all(text, 
       "(^|\\s)(@|#|https?://)\\S+", " ") %>% 
             str_replace_all("\\W+", " ")
    )
tweets


── Column specification ─────────────────────────────────────────────────────────────────────────────────
cols(
  id = col_double(),
  text = col_character()
)




id,text,has_tag,n_at,url,plain2
<dbl>,<chr>,<lgl>,<int>,<chr>,<chr>
1,RT: @john_doe https://example.com/news very interesting!,FALSE,1,https://example.com/news,RT very interesting
2,tweet with just text,FALSE,0,NA,tweet with just text
3,http://example.com/pandas #breaking #mustread,TRUE,0,http://example.com/pandas,
4,@me and @myself #selfietime,TRUE,2,NA,and


In [8]:
text = "apples, pears, oranges"
items=strsplit(text,", ", fixed=T)[[1]]
items=str_split(text,"\\p{PUNCTUATION}\\s*")[[1]]
items=str_extract_all(text,"\\p{LETTER}+")[[1]]
print(items)
joined = str_c(items, collapse=" & ")
print(joined)

[1] "apples"  "pears"   "oranges"
[1] "apples & pears & oranges"


In [9]:
tags = tweets %>% mutate(
    tag=str_extract_all(tweets$text,"(#\\w+)"))%>%
  select(id, tag)
tags_long = tags  %>% unnest(tag)
left_join(tags_long, tweets)

Joining, by = "id"



id,tag,text,has_tag,n_at,url,plain2
<dbl>,<chr>,<chr>,<lgl>,<int>,<chr>,<chr>
3,#breaking,http://example.com/pandas #breaking #mustread,TRUE,0,http://example.com/pandas,
3,#mustread,http://example.com/pandas #breaking #mustread,TRUE,0,http://example.com/pandas,
4,#selfietime,@me and @myself #selfietime,TRUE,2,NA,and


In [10]:
words = tweets %>% mutate(
    word=str_split(tweets$text, "\\W+")) %>% 
  select(id, word)
words_long = words %>% unnest(word)
head(words_long)

id,word
<dbl>,<chr>
1,RT
1,john_doe
1,https
1,example
1,com
1,news


In [11]:
words_long %>% 
  group_by(id) %>% 
  summarize(joined=str_c(word, collapse="_"))

id,joined
<dbl>,<chr>
1,RT_john_doe_https_example_com_news_very_interesting_
2,tweet_with_just_text
3,http_example_com_pandas_breaking_mustread
4,_me_and_myself_selfietime
